#### https://medium.com/mlearning-ai/fine-tuning-bert-for-tweets-classification-ft-hugging-face-8afebadd5dbf

In [1]:
from datasets import load_dataset

/home/kshitij/repos/robber/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset cha path change kr karshil
dataset=load_dataset('csv',data_files={'train':'../dataset/data.csv'})

In [3]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

In [4]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [5]:
print(tokenizer("hello"))

{'input_ids': [101, 7592, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [6]:
def tokenize_data(example):
    return tokenizer(example['tweet'],padding='max_length')

In [7]:
dataset=dataset.map(tokenize_data)

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6596
    })
})


In [9]:
print(dataset.remove_columns(['labels', 'tweet']))

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6596
    })
})


### Training

In [10]:
!pip install accelerate
from transformers import TrainingArguments

training_args=TrainingArguments("test_trainer",num_train_epochs=3)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [22]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6596
    })
})


In [25]:
#splitting the dataset 
#baad mein karunga

# Wrong order
# train_dataset=dataset['train'].shuffle(seed=10).select(range(5000))
# eval_dataset=dataset['train'].shuffle(seed=10).select(range(5000,6615))

# First select, then shuffle
train_dataset=dataset['train'].select(range(5000)).shuffle(seed=10)
eval_dataset=dataset['train'].select(range(5000, 6596)).shuffle(seed=10)


In [15]:
# only keep input ids and stuff
# train_dataset = train_dataset.remove_columns(['labels', 'tweet'])
# eval_dataset = eval_dataset.remove_columns(['labels', 'tweet'])

In [26]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['labels', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5000
})
Dataset({
    features: ['labels', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1596
})


In [27]:
from transformers import Trainer

trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
# trainer.evaluate(metric_key_prefix='test')

[Some Fix](https://discuss.huggingface.co/t/valueerror-target-size-torch-size-8-must-be-the-same-as-input-size-torch-size-8-8/12133/9)

In [20]:

trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
